# Quickstart

This notebook serves as a brief overview for getting started with pyCIMS. For this notebook to run properly, ensure:
* pyCIMS is downloaded and installed on your local machine according to the [installation instructions](../../docs/Installation.md)
* The `pyCIMS_env` conda environment has been activated according the [conda environment instructions](../../docs/WorkingWithCondaEnvironment.md)


## Import pyCIMS  & other packages

In [1]:
import pyCIMS
import pprint as pp

## Model Validator

Now that we have loaded the `pyCIMS` module we can start to use it. First, we will validate our model description using the `ModelValidator` class.

In [2]:
model_description_file = '../../pyCIMS_model_description_Alberta_Validated.xlsb'
model_validator = pyCIMS.ModelValidator(model_description_file)
model_validator.validate(verbose=True, raise_warnings=False)

0 node name/branch mismatches. 
0 references to unspecified nodes. 
0 non-root nodes are never referenced. 
0 nodes were specified but don't provide a service. 
0 nodes had invalid competition types. 
2 nodes requested services of themselves. See ModelValidator.warnings['nodes_requesting_self'] for more info
139 nodes or technologies don't request other services. See ModelValidator.warnings['nodes_no_requested_service'] for more info


For more information on using the `ModelValidator` class, checkout the [model validation tutorial](model_validation_instructions.ipynb). 

## Build the Model
There are two steps to building the model. First, we need to create a `ModelReader` object. This object reads in the model description Excel file and does some processessing to prepare the description to become a model. Then, this `ModelReader` object becomes the input to the all important `Model` object.

In [3]:
# Model Reader
model_reader = pyCIMS.ModelReader(infile=model_description_file,
                                  sheet_map={'model': 'Model'},
                                  node_col='Node')

In [4]:
# Model
model = pyCIMS.Model(model_reader)

## Run the pyCIMS Model

In [5]:
model.run(max_iterations=5, show_warnings=False)

***** ***** year: 2000 ***** *****
iter 0
***** ***** year: 2005 ***** *****


/Users/jilliana/Documents/Consulting/pyCIMS/pycims_prototype/pyCIMS/graph_utils.py:221: UserWarning: Found a Loop -- 
  warnings.warn("Found a Loop -- ")
/Users/jilliana/Documents/Consulting/pyCIMS/pycims_prototype/pyCIMS/econ.py:164: UserWarning: Discount rate_Financial has value of 0 at pyCIMS.Canada.Alberta.Residential.Buildings.Furnace.Furnace Fan -- Standard
  warnings.warn('Discount rate_Financial has value of 0 at {} -- {}'.format(node, tech))
/Users/jilliana/Documents/Consulting/pyCIMS/pycims_prototype/pyCIMS/econ.py:164: UserWarning: Discount rate_Financial has value of 0 at pyCIMS.Canada.Alberta.Residential.Buildings.Furnace.Furnace Fan -- Efficient
  warnings.warn('Discount rate_Financial has value of 0 at {} -- {}'.format(node, tech))
/Users/jilliana/Documents/Consulting/pyCIMS/pycims_prototype/pyCIMS/econ.py:164: UserWarning: Discount rate_Financial has value of 0 at pyCIMS.Canada.Alberta.Petroleum Crude.CCS -- CCS
  warnings.warn('Discount rate_Financial has value of 0 at

iter 0
iter 1
iter 2
***** ***** year: 2010 ***** *****
iter 0
iter 1
***** ***** year: 2015 ***** *****
iter 0


/Users/jilliana/Documents/Consulting/pyCIMS/pycims_prototype/pyCIMS/econ.py:164: UserWarning: Discount rate_Financial has value of 0 at pyCIMS.Canada.Alberta.Waste.Small Sites -- Small_Sites_Flaring_>_2011
  warnings.warn('Discount rate_Financial has value of 0 at {} -- {}'.format(node, tech))
/Users/jilliana/Documents/Consulting/pyCIMS/pycims_prototype/pyCIMS/econ.py:164: UserWarning: Discount rate_Financial has value of 0 at pyCIMS.Canada.Alberta.Waste.Small Sites -- Small_Sites_Electricity_Generation_>_2011
  warnings.warn('Discount rate_Financial has value of 0 at {} -- {}'.format(node, tech))
/Users/jilliana/Documents/Consulting/pyCIMS/pycims_prototype/pyCIMS/econ.py:164: UserWarning: Discount rate_Financial has value of 0 at pyCIMS.Canada.Alberta.Waste.Medium Sites -- Medium_Sites_Flaring_>_2011
  warnings.warn('Discount rate_Financial has value of 0 at {} -- {}'.format(node, tech))
/Users/jilliana/Documents/Consulting/pyCIMS/pycims_prototype/pyCIMS/econ.py:164: UserWarning: Disc

iter 1
***** ***** year: 2020 ***** *****
iter 0
iter 1
***** ***** year: 2025 ***** *****
iter 0
iter 1
***** ***** year: 2030 ***** *****
iter 0
iter 1
***** ***** year: 2035 ***** *****
iter 0
iter 1
***** ***** year: 2040 ***** *****
iter 0
iter 1
***** ***** year: 2045 ***** *****
iter 0
iter 1
***** ***** year: 2050 ***** *****
iter 0
iter 1


## Examine the Results

#### How has a fuel's price changed over the years?
This cell will print out the LCC for each year for the specified fuel. The `fuel_of_interest` variable should be the path to the fuel node of interest. 

In [6]:
fuel_of_interest = 'pyCIMS.Canada.Alberta.Electricity' # Change this line to see how other fuels change

print("----- {} -----".format(fuel_of_interest))
for year in model.years:
    fuel_name = list(model.graph.nodes[fuel_of_interest][year]['Life Cycle Cost'].keys())[0]
    lcc = model.graph.nodes[fuel_of_interest][year]['Life Cycle Cost'][fuel_name]['year_value']
    print('{}: {}'.format(year, round(lcc, 2)))


----- pyCIMS.Canada.Alberta.Electricity -----
2000: 22
2005: 64.83
2010: 62.85
2015: 64.63
2020: 43.33
2025: 48.85
2030: 48.28
2035: 46.2
2040: 43.72
2045: 41.76
2050: 36.99


#### How has marketshare changed between years?
This cell will print out the market shares for each technology for each within the specified node. The `node_of_interest` variable should be the path to the node of interest. 

In [7]:
node_of_interest = 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting' # Change this line to see other nodes

# Printing the Table Header
headers = ('Technology', 'New Market Share', 'Total Market Share')
dash = '-' * 70
cols = '{:<20}{:>25}{:>25}'
print("{}".format(node_of_interest))
print(dash)
print(cols.format(*headers))             
print(dash)

# Printing the Table Content
total_stocks = {}
for year in model.years:
    print(year)
    for tech, data in model.graph.nodes[node_of_interest][year]['technologies'].items():
        n_ms = str(round(data['new_market_share']*100)) + "%"
        t_ms = str(round(data['total_market_share']*100)) + '%'      
        print(cols.format(tech, n_ms, t_ms))             
    print(dash)



pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting
----------------------------------------------------------------------
Technology                   New Market Share       Total Market Share
----------------------------------------------------------------------
2000
Incandescent                              92%                      92%
CFL                                        8%                       8%
LED                                        0%                       0%
----------------------------------------------------------------------
2005
Incandescent                               0%                       0%
CFL                                      100%                     100%
LED                                        0%                       0%
----------------------------------------------------------------------
2010
Incandescent                               0%                       0%
CFL                                      100%                     100

#### How to find a Node's Path
This cell retrieves all of the nodes within the graph whose name contains the specified `node_name` . A Node's name is the last element in path, where elements are seperated by `.`. 

In [8]:
node_name = 'Furnace'

for n in model.graph.nodes:
    n_name = n.split('.')[-1]
    if node_name in n_name:
        print(n)


pyCIMS.Canada.Alberta.Residential.Buildings.Furnace
pyCIMS.Canada.Alberta.Residential.Buildings.Furnace.Furnace Fan
pyCIMS.Canada.Alberta.Industrial Minerals.Products.Glass.Flat.Furnaces
pyCIMS.Canada.Alberta.Industrial Minerals.Products.Glass.Containers.Furnaces
pyCIMS.Canada.Alberta.Industrial Minerals.Products.Glass.Blown Pressed.Furnaces
pyCIMS.Canada.Alberta.Industrial Minerals.Products.Glass.Fibreglass.Furnaces
pyCIMS.Canada.Alberta.Pulp and Paper.Black Liquor Service.Concentrated Black Liquor.Recovery Furnace
